In [1]:
import pandas as pd

## Lire les donnees via l'api

In [ ]:
from milestone1.question1 import get_play_by_play_season_gameType

match_type = 2

get_play_by_play_season_gameType("2015","0"+str(match_type),"milestone1/data_saved/play_by_play")
get_play_by_play_season_gameType("2016","0"+str(match_type),"milestone1/data_saved/play_by_play")
get_play_by_play_season_gameType("2017","0"+str(match_type),"milestone1/data_saved/play_by_play")
get_play_by_play_season_gameType("2018","0"+str(match_type),"milestone1/data_saved/play_by_play")
get_play_by_play_season_gameType("2019","0"+str(match_type),"milestone1/data_saved/play_by_play")

## Creer les dataframes de test et train/val

In [1]:
from milestone1.question4 import *

#les donnes des saisons régulière seulement!
directory2015 = r'data_saved/play_by_play/2015/regular'
directory2016 = r'data_saved/play_by_play/2016/regular'
directory2017 = r'data_saved/play_by_play/2017/regular'
directory2018 = r'data_saved/play_by_play/2018/regular'
directory2019 = r'data_saved/play_by_play/2019/regular'


df2015 = create_full_df(directory=directory2015)
df2016 = create_full_df(directory=directory2016)
df2017 = create_full_df(directory=directory2017)
df2018 = create_full_df(directory=directory2018)

# 2019/20 comme l' ensemble final des données test.
df2019 = create_full_df(directory=directory2019)


In [11]:
#Question 4.4 Bonus
from milestone2.question4 import *

df2015 = add_power_play(df2015, directory2015)
df2016 = add_power_play(df2016, directory2016)
df2017 = add_power_play(df2017, directory2017)
df2018 = add_power_play(df2018, directory2018)
df2019 = add_power_play(df2019, directory2019)

NameError: name 'add_power_play' is not defined

In [2]:
df15_16 = pd.concat([df2015, df2016], ignore_index=True)
df17_18 = pd.concat([df2017, df2018], ignore_index=True)

#utilise les données 2015/16 - 2018/19 de la saison régulière pour créer vos ensembles d'entraînement et de validation.
df_train_val =  pd.concat([df15_16, df17_18], ignore_index=True)

In [3]:
print(df_train_val.head())

      Game_ID  Event_ID  Period_Number Period_Time  Game_Time Shot_or_Goal  \
0  2015020001         7              1       00:51       0.85         Shot   
1  2015020001         9              1       01:05       1.80         Shot   
2  2015020001        10              1       01:06       1.10         Shot   
3  2015020001        13              1       01:53       1.88         Shot   
4  2015020001        21              1       02:37       2.61         Shot   

    Shot_Type        Shooter Team_of_Shooter       Goalie Empty_Net  \
0  Wrist Shot     Brad Boyes             TOR  Carey Price       NaN   
1   Snap Shot    Nazem Kadri             TOR  Carey Price       NaN   
2   Snap Shot    Nazem Kadri             TOR  Carey Price       NaN   
3  Wrist Shot  Daniel Winnik             TOR  Carey Price       NaN   
4      Tip-In  Joffrey Lupul             TOR  Carey Price       NaN   

  Goal_Strength  X_Coordinate  Y_Coordinate   Distance      Angle  Est_un_but  \
0           NaN        

In [7]:
print(df_train_val["Game_ID"].dtypes)

int64


## Question 4.1: Features management

In [5]:
df_train_val = df_train_val[["Game_Time","Period_Number", "X_Coordinate", "Y_Coordinate", "Distance", "Angle", "Shot_or_Goal", "pwplay_RemainingTime", "pwplay_fplayer", "pwplay_eplayer"]]

## Question 4.2: Previous elements infos

In [7]:
df_train_val["Game_Time"] = (df_train_val["Game_Time"]*60).apply(lambda x: int(x))

In [8]:
df_train_val["Pre_type"] = [np.nan] + [df_train_val.Shot_or_Goal[i-1] for i in range(1,len(df_train_val.Shot_or_Goal))]

In [9]:
df_train_val["Pre_X"] = [np.nan] + [df_train_val.X_Coordinate[i-1] for i in range(1,len(df_train_val))]

In [10]:
df_train_val["Pre_Y"] = [np.nan] + [df_train_val.Y_Coordinate[i-1] for i in range(1,len(df_train_val))]

In [11]:
df_train_val["Time_Since_LastEvnt"] = abs(df_train_val["Game_Time"].diff())

In [19]:
df_train_val["Dist_from_LastEvnt"] = [abs(np.linalg.norm(np.array([df_train_val.X_Coordinate[i], df_train_val.Y_Coordinate[i]]) - np.array([df_train_val.Pre_X[i], df_train_val.Pre_Y[i]]))) for i in range(len(df_train_val))]

In [20]:
df_train_val.head()

,Game_Time,Period_Number,X_Coordinate,Y_Coordinate,Distance,Angle,Shot_or_Goal,Pre_type,Pre_X,Pre_Y,Time_Since_LastEvnt,Dist_from_LastEvnt
0,51,1,-55.0,6.0,35.510562,9.727579,Shot,NaN,NaN,NaN,NaN,NaN
1,108,1,-79.0,0.0,11.000000,0.000000,Shot,Shot,-55.0,6.0,57.0,24.738634
2,66,1,-75.0,-1.0,15.033296,-3.814075,Shot,Shot,-79.0,0.0,42.0,4.123106
3,112,1,-55.0,-36.0,50.209561,-45.806929,Shot,Shot,-75.0,-1.0,46.0,40.311289
4,156,1,-61.0,-1.0,29.017236,-1.974934,Shot,Shot,-55.0,-36.0,44.0,35.510562


## Question 4.3: New Features

In [21]:
df_train_val["Rebond"] = [df_train_val.Shot_or_Goal[i] == df_train_val.Pre_type[i] for i in range(len(df_train_val))]

In [22]:
df_train_val["Changement_d'angle_de_tir"] = [int(df_train_val.Rebond[i]) for i in range(len(df_train_val))]

In [25]:
df_train_val["Vitesse"] = df_train_val.Dist_from_LastEvnt/df_train_val.Time_Since_LastEvnt

In [26]:
df_train_val.head()

,Game_Time,Period_Number,X_Coordinate,Y_Coordinate,Distance,Angle,Shot_or_Goal,Pre_type,Pre_X,Pre_Y,Time_Since_LastEvnt,Dist_from_LastEvnt,Rebond,Changement_d'angle_de_tir,Vitesse
0,51,1,-55.0,6.0,35.510562,9.727579,Shot,NaN,NaN,NaN,NaN,NaN,False,0,NaN
1,108,1,-79.0,0.0,11.000000,0.000000,Shot,Shot,-55.0,6.0,57.0,24.738634,True,1,0.434011
2,66,1,-75.0,-1.0,15.033296,-3.814075,Shot,Shot,-79.0,0.0,42.0,4.123106,True,1,0.098169
3,112,1,-55.0,-36.0,50.209561,-45.806929,Shot,Shot,-75.0,-1.0,46.0,40.311289,True,1,0.876332
4,156,1,-61.0,-1.0,29.017236,-1.974934,Shot,Shot,-55.0,-36.0,44.0,35.510562,True,1,0.807058
